<a href="https://colab.research.google.com/github/matthew110395/12004210_DataAnalytics/blob/main/12004210_DAOTW_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This assignment builds on the New York taxi problem identified in assignment 1, where the City of New York is looking for a way to accurately predict the number of collisions on a particular day of the week. Within this document two different types of machine learning models will be utilised to predict the number of collisions. The linear relationships identified in assignment 1 will be used to create a linear regression model. A Deep Learning Neural Network (DNN) will be used to predict the number of collisions where the relationship is not linear. 

#Imports
To prepare data for machine learning the pandas package has been used. The numpy package has been used to aid with mathematical functions.

As within part 1 of this assignment the data file containing location data exceeds the size limit for hosting within github. To overcome this the file was zipped. In order to extract the data from the zip file the zipfile package has been used.

Within this document, Tensorflow has been used for machine learning, with both a linear regression model and a Deep Neural Network model. Tensorflow version 1 is unsupported within Google Colab, therefore must be installed using the pip package manager.

Shutil has also been imported to allow for easy file management, in particular the removal of saved models.

In [1]:
import pandas as pd
import numpy as np
import zipfile


!pip install tensorflow==1.15.2
import tensorflow as tf
# needed for high-level file management
import shutil  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 36 kB/s 
     |████████████████████████████████| 503 kB 45.7 MB/s 
     |████████████████████████████████| 50 kB 3.2 MB/s 
     |████████████████████████████████| 3.8 MB 43.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=25149e31bc61c329234451f92b31bb7aff15e42d5593faf567256e7bd9c747d2
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

#Linear Regressor
Throughout assignment 1 a number of linear relationships were uncovered within the dataset. These relationsips form the basis of the linear regression models below.

A linear regressor is used to predict an output variable based on one or more input variables. REF https://www.ibm.com/uk-en/topics/linear-regression 17/11



To improve the accuracy of the model the target values are scaled or standarised. This reduces the range of collisions from 188-1161 to 0.1619... - 1, this allows for quicker training of the model. https://towardsdatascience.com/understand-data-normalization-in-machine-learning-8ff3062101f0 REF 17.11

In [2]:
SCALE_COLLISIONS=1161

##Precipition
As uncovered in assignment 1, as the volume of precipition increases, the number of collisions increase. 

The cleaned datafile produced in assignment is imported.

In [3]:
df_prcp = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean.csv', index_col=0, )
print(df_prcp[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In order to create the linear regression model, extra columns are removed to simplify the model with the aim of reducing error values within the model.

The incomplete years (2012 and 2022) are removed, along with the eronious data for 2020 and 2021.

To aid with the production of the model the target, in this case the number of collisions is moved to the end of the data table.

The summary below shows there is 2539 rows within the dataset after the data has been cleansed further, as this count exists for each column each column or predcictor contains data.

In [4]:
df_prcp = df_prcp.drop(columns=['collision_date', 'temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp = df_prcp.loc[df_prcp["year"] != 2012]
df_prcp = df_prcp.loc[df_prcp["year"] < 2020]
cols = df_prcp['NUM_COLLISIONS']
df_prcp = df_prcp.drop(columns=['NUM_COLLISIONS'])
df_prcp.insert(loc=9, column='NUM_COLLISIONS', value=cols)
print(df_prcp[:6])
df_prcp.describe()

    day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
49    4  2016   1  28  0.09    0             0                 0     0   
51    5  2014   1  17  0.00    1             0                 0     0   
54    1  2016   1  25  0.02    0             0                 0     0   
55    5  2016   1  29  0.00    0             0                 0     0   
58    5  2017   1  20  0.00    0             0                 0     0   
59    7  2013   1  13  0.01    1             0                 0     0   

    NUM_COLLISIONS  
49             681  
51             589  
54             658  
55             645  
58             605  
59             373  


,day,year,mo,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,3.998425,2015.989366,6.518708,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,599.135093
std,2.003542,1.996126,3.455211,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,100.299164
min,1.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1161.000000


To remove any bias within the dataset, it is randomly shuffled. This shuffled dataset is then split into the predictors and the target.

In [5]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp.iloc[np.random.permutation(len(df_prcp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail
1450    2  2019   5  28  0.00    1             1                 0     0
2629    6  2014   9   6  0.01    1             0                 0     0
2862    4  2018  10   4  0.04    0             0                 0     0
1884    4  2019   7   4  0.00    1             0                 0     0
3294    3  2017  11   8  0.37    0             1                 0     0
2121    4  2014   7  10  0.01    1             1                 0     0
      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
1450    2  2019   5  28  0.00    1             1                 0     0   
2629    6  2014   9   6  0.01    1             0                 0     0   
2862    4  2018  10   4  0.04    0             0                 0     0   
1884    4  2019   7   4  0.00    1             0                 0     0   
3294    3  2017  11   8  0.37    0             1                 0     0   
2121    4  2014   7  10  0.01    

In [6]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1450    654
2629    553
2862    717
1884    394
3294    662
2121    584
Name: NUM_COLLISIONS, dtype: int64


To test the accuracy/reliabilty of the trained linear regressor, the dataset has been split into a training and a testing dataset. As when compared with other machine learning datasets, this dataset is relatively small a larger portion is required for testing. 20% (511 rows) of the dataset has been reserved for testing, with the remaining 80% (2044 rows) used for training the dataset.

In [7]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 9
# Define the number of output values (targets)
noutputs = 1

2031
508


In [8]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contri

starting to train


Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 0.27354407, step = 1
INFO:tensorflow:global_step/sec: 584.21
INFO:tensorflow:loss = 0.005071222, step = 101 (0.174 sec)
INFO:tensorflow:global_step/sec: 762.361
INFO:tensorflow:loss = 0.0070082378, step = 201 (0.131 sec)
INFO:tensorflow:global_step/sec: 1002.11
INFO:tensorflow:loss = 0.009041734, step = 301 (0.098 sec)
INFO:tensorflow:global_step/sec: 979.367
INFO:tensorflow:loss = 0.0058775144, step = 401 (0.102 sec)
INFO:tensorflow:global_step

LinearRegression has RMSE of 93.88877093431789
Just using average = 598.9172821270311 has RMSE of 99.19843127180098


A number of learning rates between 0.0000001 and 0.1 were been used to determine a suitable learning rate for the dataset. As the learning rate decreases the the overall time to train the dataset increases. REF https://towardsdatascience.com/understanding-learning-rates-and-how-it-improves-performance-in-deep-learning-d0d4059c1c10 17/11

In [9]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_prcp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3769ba2d10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_prc

508
508
[0.56210583 0.53232276 0.5297852  0.56320906 0.51793087 0.54421693
 0.5189606  0.4885822  0.5629493  0.5060771  0.53323096 0.51482785
 0.5022705  0.5319142  0.47829488 0.53492403 0.5072606  0.576366
 0.53238267 0.53261167 0.52869445 0.5173587  0.52983093 0.49778187
 0.5406675  0.5183686  0.5374979  0.5580212  0.56952655 0.5598694
 0.55980873 0.56438035 0.47322932 0.5438664  0.49622232 0.51793635
 0.51902276 0.51678294 0.5694296  0.49900186 0.5879601  0.53330916
 0.52503353 0.5245062  0.5566466  0.5495427  0.54309195 0.49033862
 0.5730556  0.4935801  0.50969756 0.5605255  0.4862097  0.55127734
 0.53353316 0.5706239  0.5781539  0.5427215  0.5140844  0.54068124
 0.5248328  0.50971925 0.5672169  0.5089115  0.5005322  0.48349398
 0.57070464 0.504525   0.532006   0.51298404 0.51389897 0.5258202
 0.5018219  0.48371303 0.5396503  0.52186316 0.5510464  0.5238042
 0.5300965  0.5355549  0.53471196 0.5463198  0.5456771  0.5771824
 0.50043696 0.54656464 0.50638044 0.52123314 0.47713768 0.54

Two main tests have been applied to the model. The Route Mean Squared Error (RMSE) and a comparison between the target values in the testing dataset and the predicted values using the predictors in the testing dataset.

Due to the nature of machine learning, different results are achieved with each run of the model.

Predominantly the RMSE of the model is lower than that of the average. This indicates that the linear regression model makes more accurate predictions when compared to using just the average. The error rate of the model is aprocimately -2% when comparing the predicited and actual results.

##Dew Point (dewp)
A relationship between dew point and the number of collisions was also uncovered in assignment 1. This linear relationship suggests that as the dew point increases the number of collisions increase. 

The process to produce the model follows the same process as the precipitation model above where the data is loaded, further cleaned, shuffled, split into a train and test dataset prior to training and tesing the model.

In [88]:
df_dewp = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean.csv', index_col=0, )
print(df_dewp[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [89]:
df_dewp = df_dewp.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp = df_dewp.loc[df_dewp["year"] != 2012]
df_dewp = df_dewp.loc[df_dewp["year"] < 2020]
cols = df_dewp['NUM_COLLISIONS']
df_dewp = df_dewp.drop(columns=['NUM_COLLISIONS'])
df_dewp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_dewp[:6])
df_dewp.describe()

    day  year  mo  da  dewp  NUM_COLLISIONS
49    4  2016   1  28  24.4             681
51    5  2014   1  17  35.8             589
54    1  2016   1  25  21.2             658
55    5  2016   1  29  36.8             645
58    5  2017   1  20  32.5             605
59    7  2013   1  13  44.9             373


,day,year,mo,da,dewp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.998434,2015.999217,6.524070,15.723679,44.163170,599.109980
std,2.000391,2.000000,3.449676,8.801271,16.995303,100.277185
min,1.000000,2013.000000,1.000000,1.000000,-6.700000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,32.150000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,45.300000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,58.500000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,74.100000,1161.000000


In [90]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp.iloc[np.random.permutation(len(df_dewp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  dewp
966     7  2015   4  26  34.0
1292    4  2019   5  16  47.6
3096    2  2016  11   1  37.1
1472    3  2016   5  18  45.2
2339    3  2015   8  26  68.1
2121    4  2014   7  10  66.1
      day  year  mo  da  dewp  NUM_COLLISIONS
966     7  2015   4  26  34.0             477
1292    4  2019   5  16  47.6             642
3096    2  2016  11   1  37.1             732
1472    3  2016   5  18  45.2             676
2339    3  2015   8  26  68.1             606
2121    4  2014   7  10  66.1             584


In [92]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

966     477
1292    642
3096    732
1472    676
2339    606
2121    584
Name: NUM_COLLISIONS, dtype: int64


In [93]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [94]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d45d8d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.


starting to train


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 0.28104508, step = 1
INFO:tensorflow:global_step/sec: 574.815
INFO:tensorflow:loss = 0.007068947, step = 101 (0.176 sec)
INFO:tensorflow:global_step/sec: 756.946
INFO:tensorflow:loss = 0.007001254, step = 201 (0.132 sec)
INFO:tensorflow:global_step/sec: 896.518
INFO:tensorflow:loss = 0.0073363464, step = 301 (0.116 sec)
INFO:tensorflow:global_step/sec: 893.974
INFO:tensorflow:loss = 0.0068931864, step = 401 (0.108 sec)
INFO:tensorflow:global_step/sec: 1042.87
INFO:tensorflow:loss = 0.0068940776, step = 501 (0.097 sec)
INFO:tensorflow:global_step/sec: 1041.25
INFO:tensorflow:loss = 0.0067550116, step = 601 (0.094 sec)
INFO:tensorflow:global_step/sec: 1012.67
INFO:tensorflow:loss = 0.008446421, step = 701 (0.101 sec)
INFO:tensorflow:global_step/sec:

LinearRegression has RMSE of 89.12176891716852
Just using average = 598.116927592955 has RMSE of 96.06957549415907


In [95]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_dewp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376db950d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_dew

511
511
[0.5438962  0.4851647  0.5026894  0.52537984 0.5434492  0.50346303
 0.56172353 0.5625193  0.49681592 0.57240576 0.5254114  0.488935
 0.56094795 0.52711546 0.50265723 0.5588435  0.5061072  0.5043206
 0.5340504  0.5491598  0.47387102 0.46157077 0.46813536 0.49736303
 0.47743794 0.5085834  0.5020151  0.48627532 0.4823827  0.49648526
 0.48104838 0.4809948  0.48267835 0.5071648  0.5249856  0.5351226
 0.5253108  0.5200598  0.5222175  0.45756918 0.5137877  0.5308395
 0.50849324 0.5040166  0.4915178  0.5117137  0.49590725 0.5124543
 0.5316618  0.45008358 0.54385465 0.5225567  0.5478861  0.5719523
 0.45546645 0.5359801  0.4767777  0.5228509  0.5373077  0.48635948
 0.4684854  0.49111587 0.4698773  0.46211514 0.49908686 0.4597421
 0.50924057 0.47888893 0.531697   0.5389135  0.5337841  0.47731683
 0.5660528  0.5369272  0.5192258  0.49174914 0.55625176 0.49475342
 0.5062241  0.5609131  0.49400234 0.46765593 0.5524619  0.4711879
 0.48522684 0.5722692  0.47384644 0.53939754 0.48984057 0.50530

The RMSE for the dewp model is generally slightly lower in comparison to the model procued for precipitation. As the RMSE for the model is lower than the RMSE of the mean, it can be argued that the model has a higher level of accuracy in comparision to the using the mean.

##Visibility (visib)
A further relationship was uncovered between visibilty and the number of collisions. This is a negative linear relationship where as the visibility increases the number of collisions decrease. 

The process to produce the model follows the same process as above where the data is loaded, further cleaned, shuffled, split into a train and test dataset prior to training and tesing the model.

In [97]:
df_visib = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/coldata.csv', index_col=0, )
print(df_visib[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [98]:
df_visib = df_visib.drop(columns=['collision_date', 'temp', 'prcp', 'slp','dewp','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_visib = df_visib.loc[df_visib["year"] != 2012]
df_visib = df_visib.loc[df_visib["year"] < 2020]
cols = df_visib['NUM_COLLISIONS']
df_visib = df_visib.drop(columns=['NUM_COLLISIONS'])
df_visib.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_visib[:6])
df_visib.describe()

    day  year  mo  da  visib  NUM_COLLISIONS
49    4  2016   1  28   10.0             681
51    5  2014   1  17    6.7             589
54    1  2016   1  25   10.0             658
55    5  2016   1  29   10.0             645
58    5  2017   1  20   10.0             605
59    7  2013   1  13    4.3             373


,day,year,mo,da,visib,NUM_COLLISIONS
count,2556.000000,2556.0,2556.000000,2556.000000,2556.000000,2556.000000
mean,3.999218,2016.0,6.524257,15.725743,8.295618,599.118936
std,2.000391,2.0,3.449013,8.800168,2.207870,100.258581
min,1.000000,2013.0,1.000000,1.000000,0.200000,188.000000
25%,2.000000,2014.0,4.000000,8.000000,7.100000,531.000000
50%,4.000000,2016.0,7.000000,16.000000,9.400000,602.000000
75%,6.000000,2018.0,10.000000,23.000000,10.000000,665.000000
max,7.000000,2019.0,12.000000,31.000000,10.000000,1161.000000


In [99]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_visib.iloc[np.random.permutation(len(df_visib))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  visib
1615    5  2014   6  20    9.8
385     7  2019   2  17   10.0
3338    7  2016  11  20    5.5
395     7  2016   2  21    8.4
808     4  2015   3  12   10.0
805     7  2019   3  24   10.0
      day  year  mo  da  visib  NUM_COLLISIONS
1615    5  2014   6  20    9.8             765
385     7  2019   2  17   10.0             439
3338    7  2016  11  20    5.5             529
395     7  2016   2  21    8.4             481
808     4  2015   3  12   10.0             586
805     7  2019   3  24   10.0             466


In [100]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1615    765
385     439
3338    529
395     481
808     586
805     466
Name: NUM_COLLISIONS, dtype: int64


In [101]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
512


In [106]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_visib', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_visib', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d437490>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_visib', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_visib/model.ckpt.
INFO:tensorflow:loss = 0.28360546, step = 1
INFO:tensorflow:global_step/sec: 838.916
INFO:tensorflow:loss = 0.010178417, step = 101 (0.125 sec)
INFO:tensorflow:global_step/sec: 972.157
INFO:tensorflow:loss = 0.007864419, step = 201 (0.099 sec)
INFO:tensorflow:global_step/sec: 941.387
INFO:tensorflow:loss = 0.006883328, step = 301 (0.109 sec)
INFO:tensorflow:global_step/sec: 889.726
INFO:tensorflow:loss = 0.0055556484, step = 401 (0.112 sec)
INFO:tensorflow:global_step/sec: 977.529
INFO:tensorflow:loss = 0.0057206037, step = 501 (0.099 sec)
INFO:tensorflow:global_step/sec: 883.836
INFO:tensorflow:loss = 0.00718684, step = 601 (0.116 sec)
INFO:tensorflow:global_step/sec: 944.769
INFO:tensorflow:loss = 0.0074655795, step = 701 (

LinearRegression has RMSE of 93.73226959331255
Just using average = 599.7318982387476 has RMSE of 99.26465090834728


In [107]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_visib', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d4e6350>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_visib', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_vi

512
512
[0.5504391  0.55971545 0.5072935  0.5370538  0.53530693 0.52788585
 0.5033492  0.4765762  0.50716877 0.488393   0.57567686 0.49013257
 0.52894205 0.52244145 0.44171983 0.44245112 0.4876567  0.44718105
 0.5308005  0.4446873  0.5322599  0.5281426  0.5290186  0.5027014
 0.51841724 0.51194924 0.5305418  0.4955938  0.53985393 0.5401778
 0.50576967 0.5347008  0.50672823 0.48755383 0.47575164 0.5361112
 0.47728527 0.49773952 0.4604193  0.4720142  0.47537768 0.5618585
 0.508723   0.55307496 0.5005889  0.5239452  0.5207383  0.49846473
 0.51298594 0.46904385 0.47171193 0.53308374 0.5536526  0.49934363
 0.45558247 0.5406828  0.53935236 0.47306567 0.5467636  0.51331216
 0.51389056 0.5355613  0.49564394 0.53242517 0.5099678  0.47282064
 0.5233621  0.5047874  0.4623144  0.47548264 0.55878884 0.49825191
 0.48070648 0.5313772  0.519996   0.51458204 0.5574097  0.5179419
 0.4957683  0.52014995 0.5503821  0.46177587 0.536709   0.5185787
 0.54585165 0.55909115 0.4970733  0.52084327 0.48388833 0.55

As the difference RMSE between the mean and the model is less, it is arguable that the model is not as accurate when predicting the number of collisions given the level of visibity as a predictor.

Although the RMSE value indicates a weaker model, the error rate is lower indicating that there is a significant error increasing the RMSE.

##Sea Level Pressure (SLP) -REMOVE?


In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [18]:
df_slp = df.drop(columns=['collision_date', 'temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp = df_slp.loc[df_slp["year"] != 2012]
df_slp = df_slp.loc[df_slp["year"] < 2020]
cols = df_slp['NUM_COLLISIONS']
df_slp = df_slp.drop(columns=['NUM_COLLISIONS'])
df_slp.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_slp[:6])
df_slp.describe()

    day  year  mo  da     slp  NUM_COLLISIONS
49    4  2016   1  28  1016.1             681
51    5  2014   1  17  1014.8             589
54    1  2016   1  25  1021.4             658
55    5  2016   1  29   999.4             645
58    5  2017   1  20  1015.5             605
59    7  2013   1  13  1020.7             373


,day,year,mo,da,slp,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,3.999217,2016.000391,6.522114,15.719765,1016.777221,599.147162
std,2.000783,2.000294,3.447986,8.796698,7.628429,100.268048
min,1.000000,2013.000000,1.000000,1.000000,989.500000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,1012.200000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,1016.700000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,1021.700000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,1044.200000,1161.000000


In [19]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp.iloc[np.random.permutation(len(df_slp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da     slp
1403    4  2019   5  23  1022.9
3128    5  2016  11  25  1023.9
129     4  2014   1  30  1027.2
727     4  2015   3  26  1016.9
2842    2  2015  10  13  1003.5
1800    5  2017   6  16  1018.9
      day  year  mo  da     slp  NUM_COLLISIONS
1403    4  2019   5  23  1022.9             722
3128    5  2016  11  25  1023.9             455
129     4  2014   1  30  1027.2             657
727     4  2015   3  26  1016.9             525
2842    2  2015  10  13  1003.5             724
1800    5  2017   6  16  1018.9             657


In [20]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1403    722
3128    455
129     657
727     525
2842    724
1800    657
Name: NUM_COLLISIONS, dtype: int64


In [21]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2044
511


In [22]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376dae1890>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 0.26518497, step = 1
INFO:tensorflow:global_step/sec: 803.716
INFO:tensorflow:loss = 0.0068272715, step = 101 (0.130 sec)
INFO:tensorflow:global_step/sec: 965.684
INFO:tensorflow:loss = 0.006221723, step = 201 (0.103 sec)
INFO:tensorflow:global_step/sec: 922.837
INFO:tensorflow:loss = 0.008020048, step = 301 (0.107 sec)
INFO:tensorflow:global_step/sec: 994.413
INFO:tensorflow:loss = 0.0055495854, step = 401 (0.102 sec)
INFO:tensorflow:global_step/sec: 1009.16
INFO:tensorflow:loss = 0.007677032, step = 501 (0.098 sec)
INFO:tensorflow:global_step/sec: 855.286
INFO:tensorflow:loss = 0.004667553, step = 601 (0.120 sec)
INFO:tensorflow:global_step/sec: 983.237
INFO:tensorflow:loss = 0.0077134697, step = 701 (0

LinearRegression has RMSE of 108.62709815916816
Just using average = 598.8703522504892 has RMSE of 98.99650903705167


In [23]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_slp', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376dae5a90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_slp/

511
511
[0.47726688 0.48780158 0.522086   0.49973044 0.4978588  0.5165646
 0.44908002 0.4564626  0.47302002 0.47140738 0.46855244 0.4466032
 0.47461522 0.5117271  0.5168502  0.5141059  0.4404212  0.48614636
 0.51015514 0.477686   0.40357786 0.46565765 0.468483   0.41868368
 0.43016425 0.5318547  0.5164944  0.49383643 0.50359315 0.488932
 0.48779833 0.42585146 0.51008487 0.48235023 0.43231213 0.42731687
 0.5124463  0.47948673 0.46733722 0.48665735 0.50703335 0.43136793
 0.4666833  0.44735178 0.5113628  0.44484374 0.46621126 0.41841972
 0.51697284 0.46732652 0.46199858 0.4679905  0.49869782 0.41903698
 0.50828815 0.468562   0.43574688 0.42282537 0.45909116 0.43675506
 0.4719419  0.4227706  0.47365773 0.4684898  0.43784475 0.457773
 0.43989468 0.51391447 0.44584534 0.45944256 0.5285387  0.45349595
 0.4289899  0.52143    0.48769963 0.50172585 0.48161307 0.51627636
 0.5176956  0.43718195 0.48795906 0.4930749  0.5087599  0.47241846
 0.46075448 0.43325692 0.5088914  0.48820147 0.49211678 0.53

##Gust - Remove

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean.csv', index_col=0, )
print(df[:6])

    day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
3     5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
11    3  2020   1  15     2020-01-15             508  43.9  38.3  1019.4   
12    5  2021   1   1     2021-01-01             257  39.6  29.3  1029.3   
14    2  2022   1  25     2022-01-25             235  41.6  31.8  1013.2   
18    7  2021   1   3     2021-01-03             186  41.1  32.3  1018.0   
19    4  2020   1   2     2020-01-02             413  39.6  28.9  1011.8   

    visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
3    10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
11    8.2  ...  51.1  35.1  0.02  999.9    1             0                 0   
12   10.0  ...  54.0  33.1  0.13  999.9    0             0                 0   
14   10.0  ...  48.9  30.0  0.00  999.9    0             0                 0   
18   10.0  ...  53.1  39.0  0.00  999.9    0             0         

In [25]:
df_gust = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust = df_gust.loc[df_gust["year"] != 2012]
df_gust = df_gust.loc[df_gust["year"] < 2020]
cols = df_gust['NUM_COLLISIONS']
df_gust = df_gust.drop(columns=['NUM_COLLISIONS'])
df_gust.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_gust[:6])
df_gust.describe()

    day  year  mo  da  gust  NUM_COLLISIONS
74    7  2016   1  17  18.1             451
76    4  2014   1   9  20.0             561
79    6  2019   1  19  21.0             479
80    7  2015   1  11  17.1             341
83    4  2015   1  29  20.0             519
85    7  2019   1  13  15.9             374


,day,year,mo,da,gust,NUM_COLLISIONS
count,1629.000000,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000
mean,4.024555,2015.91283,6.278699,15.702885,27.511602,596.513198
std,1.989070,2.01341,3.747683,8.667634,7.366770,104.479660
min,1.000000,2013.00000,1.000000,1.000000,14.000000,188.000000
25%,2.000000,2014.00000,3.000000,8.000000,22.000000,526.000000
50%,4.000000,2016.00000,6.000000,16.000000,26.000000,597.000000
75%,6.000000,2018.00000,10.000000,23.000000,31.100000,663.000000
max,7.000000,2019.00000,12.000000,31.000000,71.100000,1161.000000


In [26]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust.iloc[np.random.permutation(len(df_gust))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  gust
2982    5  2013  10  11  26.0
2581    1  2013   9  16  15.0
562     5  2013   2   1  38.1
3543    6  2018  12  15  27.0
1508    1  2016   5  16  36.9
3254    6  2018  11  17  26.0
      day  year  mo  da  gust  NUM_COLLISIONS
2982    5  2013  10  11  26.0             621
2581    1  2013   9  16  15.0             570
562     5  2013   2   1  38.1             574
3543    6  2018  12  15  27.0             628
1508    1  2016   5  16  36.9             742
3254    6  2018  11  17  26.0             559


In [27]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2982    621
2581    570
562     574
3543    628
1508    742
3254    559
Name: NUM_COLLISIONS, dtype: int64


In [28]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

1303
326


In [29]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d9be9d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 0.27558836, step = 1
INFO:tensorflow:global_step/sec: 903.246
INFO:tensorflow:loss = 0.007470894, step = 101 (0.113 sec)
INFO:tensorflow:global_step/sec: 828.447
INFO:tensorflow:loss = 0.0062682787, step = 201 (0.122 sec)
INFO:tensorflow:global_step/sec: 1004.84
INFO:tensorflow:loss = 0.0064116335, step = 301 (0.098 sec)
INFO:tensorflow:global_step/sec: 1022.8
INFO:tensorflow:loss = 0.007871846, step = 401 (0.098 sec)
INFO:tensorflow:global_step/sec: 1062.38
INFO:tensorflow:loss = 0.007480868, step = 501 (0.101 sec)
INFO:tensorflow:global_step/sec: 891.663
INFO:tensorflow:loss = 0.006258694, step = 601 (0.108 sec)
INFO:tensorflow:global_step/sec: 961.035
INFO:tensorflow:loss = 0.010331571, step = 701 (0.

LinearRegression has RMSE of 96.1909809048557
Just using average = 597.1795855717575 has RMSE of 104.64943497722479


In [30]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_gust', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d886f50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_gus

326
326
[0.53027797 0.55667156 0.5268498  0.4796487  0.4919054  0.45642954
 0.54545987 0.48299107 0.50824106 0.46172765 0.4393457  0.5505935
 0.47052285 0.47876197 0.5632546  0.4459617  0.51585454 0.49418178
 0.49783853 0.49893972 0.52450716 0.5533219  0.48155633 0.4886029
 0.505243   0.48405573 0.53516006 0.5608611  0.4821213  0.46209013
 0.5043734  0.5131686  0.46321678 0.45029688 0.560638   0.56349164
 0.4477465  0.52382034 0.5538842  0.5271109  0.5515485  0.4847281
 0.5729917  0.48477197 0.5097766  0.47860855 0.522906   0.51747274
 0.5238774  0.5658221  0.5389643  0.44475475 0.5190162  0.47534493
 0.5306313  0.5361755  0.4994357  0.48724335 0.49996743 0.5183902
 0.5628576  0.52602375 0.52806336 0.49655616 0.49452692 0.48214036
 0.5811218  0.5414016  0.5192875  0.57000774 0.49954674 0.5249707
 0.5323206  0.48616177 0.5296994  0.49164206 0.5160071  0.502125
 0.4567406  0.4957736  0.5515521  0.56200147 0.5380638  0.45850673
 0.5174355  0.50018156 0.5272658  0.49663797 0.55507964 0.484

##Maximum Sustained Wind Speed (Mxpsd) -Remove

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [32]:
df_mxpsd = df.drop(columns=['collision_date', 'temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','gust','dewp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] != 2012]
df_mxpsd = df_mxpsd.loc[df_mxpsd["year"] < 2020]
cols = df_mxpsd['NUM_COLLISIONS']
df_mxpsd = df_mxpsd.drop(columns=['NUM_COLLISIONS'])
df_mxpsd.insert(loc=5, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd[:6])
df_mxpsd.describe()

    day  year  mo  da  mxpsd  NUM_COLLISIONS
49    4  2016   1  28    8.9             681
51    5  2014   1  17    8.9             589
54    1  2016   1  25    8.9             658
55    5  2016   1  29    9.9             645
58    5  2017   1  20    9.9             605
59    7  2013   1  13    9.9             373


,day,year,mo,da,mxpsd,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,3.999608,2016.001567,6.520564,15.737172,17.240110,599.033686
std,2.001469,2.000587,3.449204,8.797367,5.858333,100.284761
min,1.000000,2013.000000,1.000000,1.000000,5.100000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,13.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,15.900000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,20.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,49.000000,1161.000000


In [33]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd.iloc[np.random.permutation(len(df_mxpsd))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  mxpsd
3592    7  2018  12  23   20.0
3649    4  2014  12  11   26.0
1905    6  2018   7  14    9.9
2353    3  2016   8   3   14.0
3427    7  2019  12  29   11.1
754     7  2019   3  31   18.1
      day  year  mo  da  mxpsd  NUM_COLLISIONS
3592    7  2018  12  23   20.0             505
3649    4  2014  12  11   26.0             669
1905    6  2018   7  14    9.9             577
2353    3  2016   8   3   14.0             687
3427    7  2019  12  29   11.1             384
754     7  2019   3  31   18.1             450


In [34]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

3592    505
3649    669
1905    577
2353    687
3427    384
754     450
Name: NUM_COLLISIONS, dtype: int64


In [35]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

2042
511


In [36]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', optimizer=tf.train.AdamOptimizer(learning_rate=0.00001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d9b71d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 0.26268768, step = 1
INFO:tensorflow:global_step/sec: 892.168
INFO:tensorflow:loss = 0.006546222, step = 101 (0.114 sec)
INFO:tensorflow:global_step/sec: 823.994
INFO:tensorflow:loss = 0.007897538, step = 201 (0.124 sec)
INFO:tensorflow:global_step/sec: 946.597
INFO:tensorflow:loss = 0.0066778315, step = 301 (0.106 sec)
INFO:tensorflow:global_step/sec: 908.81
INFO:tensorflow:loss = 0.0064044576, step = 401 (0.114 sec)
INFO:tensorflow:global_step/sec: 941.345
INFO:tensorflow:loss = 0.008723834, step = 501 (0.101 sec)
INFO:tensorflow:global_step/sec: 1001.9
INFO:tensorflow:loss = 0.005981509, step = 601 (0.099 sec)
INFO:tensorflow:global_step/sec: 979.405
INFO:tensorflow:loss = 0.005915991, step = 701 (0.

LinearRegression has RMSE of 95.30998043856992
Just using average = 598.2497551420176 has RMSE of 101.1332211003221


In [37]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_mxpsd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d7b3fd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_mx

511
511


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.46929494 0.5535782  0.504074   0.50088716 0.54088485 0.52802414
 0.49843335 0.47779536 0.46176493 0.46753922 0.46982074 0.55190325
 0.5332394  0.48021474 0.57670665 0.5505148  0.469271   0.5289351
 0.5497735  0.5438908  0.5024241  0.5511826  0.46666533 0.4874362
 0.49522007 0.5541924  0.54036695 0.52826285 0.46978033 0.4796929
 0.5125896  0.5242982  0.54819345 0.4941268  0.54023015 0.55189484
 0.50629956 0.51655275 0.5460638  0.54543924 0.49801502 0.45472208
 0.5175781  0.5161395  0.5336684  0.52472425 0.54972374 0.49095833
 0.4694322  0.56089324 0.5287943  0.56115973 0.5616587  0.47398034
 0.50847274 0.52560717 0.5587235  0.5016224  0.49427867 0.53114593
 0.49907002 0.5219892  0.53754854 0.51120335 0.55048096 0.5239389
 0.52057016 0.49541116 0.5153777  0.5062313  0.5143223  0.5338858
 0.5157742  0.52124745 0.53872955 0.55237836 0.5151989  0.48618677
 0.5027288  0.4810264  0.5093205  0.5363649  0.5293767  0.5532737
 0.55572975 0.50312304 0.5529075  0.54079115 0.51762027 0.5269719
 0

#Deep Learning Neural Network (DNN)
Although the primary outcome of assignment 1 was uncovering linear relationships, it was also shown there was more complex relationships which cannot be predicted using a linear regressor. In this case a Deep Learning Neural Network (DNN) can be used.

A Deep Learning Neural Network (DNN) is a form of unsupervised learning, where a number of hidden layers containg nodes are used to uncover non linear relationships. Karhunen, Raiko and Cho (2015) infer that deep learning neural networks work in a similar way to the human brain. REF https://doi.org/10.1016/B978-0-12-802806-3.00007-5 19/11 This is where both the relationship between the input and output data is explored, as well as the relationship between the underying data.

####Precipition (prcp)

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [39]:
df_prcp_dnn = df.drop(columns=['temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] != 2012]
df_prcp_dnn = df_prcp_dnn.loc[df_prcp_dnn["year"] < 2020]
cols = df_prcp_dnn['NUM_COLLISIONS']
df_prcp_dnn = df_prcp_dnn.drop(columns=['NUM_COLLISIONS'])
df_prcp_dnn.insert(loc=25, column='NUM_COLLISIONS', value=cols)
print(df_prcp_dnn[:6])
df_prcp_dnn.describe()

    year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  Dec  \
49  2016  28  0.09    0             0                 0     0    0    0    0   
51  2014  17  0.00    1             0                 0     0    0    0    0   
54  2016  25  0.02    0             0                 0     0    0    0    0   
55  2016  29  0.00    0             0                 0     0    0    0    0   
58  2017  20  0.00    0             0                 0     0    0    0    0   
59  2013  13  0.01    1             0                 0     0    0    0    0   

    ...  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49  ...    0    0    0    0    0    0    0    0    1             681  
51  ...    0    0    0    0    0    0    1    0    0             589  
54  ...    0    0    0    0    0    1    0    0    0             658  
55  ...    0    0    0    0    0    0    1    0    0             645  
58  ...    0    0    0    0    0    0    1    0    0             605  
59  ...    0 

,year,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,Apr,Aug,Dec,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,...,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,2015.989366,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,0.082316,0.083497,0.085467,...,0.082710,0.085467,0.079953,0.142970,0.143364,0.143757,0.142182,0.142576,0.142182,599.135093
std,1.996126,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,0.274899,0.276687,0.279630,...,0.275497,0.279630,0.271273,0.350111,0.350512,0.350913,0.349305,0.349709,0.349305,100.299164
min,2013.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [40]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp_dnn.iloc[np.random.permutation(len(df_prcp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  Apr  Aug  \
1739  2015   9  0.00    0             1                 0     0    0    0   
3211  2017  18  0.00    0             0                 0     0    0    0   
1000  2014  27  0.37    0             1                 0     0    1    0   
3119  2014  12  0.00    1             0                 0     0    0    0   
143   2016  26  0.00    0             0                 0     0    0    0   
270   2019  20  0.61    1             1                 0     0    0    0   

      Dec  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1739    0  ...    0    0    0    0    1    0    0    0    0    0  
3211    0  ...    0    1    0    0    0    0    0    0    0    0  
1000    0  ...    0    0    0    0    0    1    0    0    0    0  
3119    0  ...    0    1    0    0    0    0    0    0    1    0  
143     0  ...    0    0    0    0    1    0    0    0    0    0  
270     0  ...    0    0    0    0    0    1    0    0    

In [41]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1739    652
3211    629
1000    449
3119    546
143     734
270     411
Name: NUM_COLLISIONS, dtype: int64


In [42]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

25


In [43]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_prcp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d87d290>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}
Instructions for updating:
Please use `layer.__call__` method instead.


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_prcp/model.ckpt.
INFO:tensorflow:loss = 798834.44, step = 1
INFO:tensorflow:global_step/sec: 370.077
INFO:tensorflow:loss = 1.2682004, step = 101 (0.273 sec)
INFO:tensorflow:global_step/sec: 484.386
INFO:tensorflow:loss = 0.011474724, step = 201 (0.208 sec)
INFO:tensorflow:global_step/sec: 473.048
INFO:tensorflow:loss = 0.015397187, step = 301 (0.213 sec)
INFO:tensorflow:global_step/sec: 475.113
INFO:tensorflow:loss = 0.013880731, step = 401 (0.211 sec)
INFO:tensorflow:global_step/sec: 473.828
INFO:tensorflow:loss = 0.0142975, step = 501 (0.212 sec)
INFO:tensorflow:global_step/sec: 463.716
INFO:tensorflow:loss = 0.013032053, step = 601 (0.212 sec)
INFO:tensorflow:global_step/sec: 617.908
INFO:tensorflow:loss = 0.013579579, step = 701 (0.164 sec)


DNNRegression has RMSE of 93.74234685742437
Just using average = 598.4101427868045 has RMSE of 101.82952104666464


In [44]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_prcp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d56c850>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_prcp', '_session_creation_timeout_secs': 7200}


508
508


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_prcp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5060923  0.538975   0.47282818 0.48340252 0.46773174 0.43765667
 0.5395014  0.5066798  0.5112803  0.5418589  0.5222285  0.4821284
 0.42439678 0.41203716 0.52383065 0.4757655  0.4636729  0.36477306
 0.5360911  0.4059184  0.53789926 0.5042994  0.4580424  0.5297816
 0.44667462 0.49606732 0.54194283 0.4320872  0.53397775 0.50532174
 0.47484997 0.5465815  0.5064738  0.5018656  0.45395306 0.47741345
 0.5022776  0.38008526 0.47395733 0.47222546 0.4066508  0.4102519
 0.48615673 0.50441384 0.5299494  0.508549   0.47113445 0.45838574
 0.364445   0.40471295 0.49804333 0.40429333 0.46783856 0.4934428
 0.5055506  0.48051098 0.5210917  0.44837597 0.54478097 0.4823039
 0.3907664  0.46537426 0.48817852 0.47753552 0.5804026  0.48899487
 0.39472607 0.4747813  0.49580792 0.48545483 0.49438885 0.5286524
 0.49016216 0.4905055  0.48656872 0.50091195 0.4608195  0.4020503
 0.5043452  0.48201397 0.4982722  0.50270486 0.45720318 0.5168574
 0.52118325 0.47367504 0.47772625 0.4924586  0.5228312  0.48710278
 0.

##Dew Point (dewp)


In [45]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/dewp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [46]:
df_dewp_dnn = df.drop(columns=['temp', 'prcp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] != 2012]
df_dewp_dnn = df_dewp_dnn.loc[df_dewp_dnn["year"] < 2020]
cols = df_dewp_dnn['NUM_COLLISIONS']
df_dewp_dnn = df_dewp_dnn.drop(columns=['NUM_COLLISIONS'])
df_dewp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_dewp_dnn[:6])
df_dewp_dnn.describe()

    year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  24.4    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  35.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  21.2    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29  36.8    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  32.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  44.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,dewp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2015.999217,15.723679,44.163170,0.082192,0.084932,0.084932,0.077104,0.084932,0.084540,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142857,599.109980
std,2.000000,8.801271,16.995303,0.274710,0.278834,0.278834,0.266808,0.278834,0.278251,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349996,100.277185
min,2013.000000,1.000000,-6.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,32.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,45.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,58.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,74.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [47]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_dewp_dnn.iloc[np.random.permutation(len(df_dewp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  dewp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
1792  2013  19  54.4    0    0    0    0    0    0    1  ...    0    0    0   
1078  2016   6  15.3    1    0    0    0    0    0    0  ...    0    0    0   
2670  2016  27  59.8    0    0    0    0    0    0    0  ...    0    0    0   
2411  2018  23  61.6    0    1    0    0    0    0    0  ...    0    0    0   
1104  2017  22  44.5    1    0    0    0    0    0    0  ...    0    0    0   
763   2013  13  41.1    0    0    0    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1792    0    0    0    0    0    1    0  
1078    0    0    0    0    0    1    0  
2670    1    1    0    0    0    0    0  
2411    0    0    0    0    0    0    1  
1104    0    0    0    0    0    0    0  
763     0    0    0    0    0    1    0  

[6 rows x 21 columns]


In [48]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1792    622
1078    628
2670    659
2411    610
1104    551
763     636
Name: NUM_COLLISIONS, dtype: int64


In [49]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [50]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_dewp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d471450>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_dewp/model.ckpt.
INFO:tensorflow:loss = 1425.2727, step = 1
INFO:tensorflow:global_step/sec: 542.081
INFO:tensorflow:loss = 0.14055178, step = 101 (0.189 sec)
INFO:tensorflow:global_step/sec: 718.322
INFO:tensorflow:loss = 0.014738771, step = 201 (0.139 sec)
INFO:tensorflow:global_step/sec: 696.321
INFO:tensorflow:loss = 0.009598495, step = 301 (0.142 sec)
INFO:tensorflow:global_step/sec: 716.42
INFO:tensorflow:loss = 0.018171396, step = 401 (0.141 sec)
INFO:tensorflow:global_step/sec: 660.589
INFO:tensorflow:loss = 0.014854988, step = 501 (0.151 sec)
INFO:tensorflow:global_step/sec: 661.739
INFO:tensorflow:loss = 0.0065168226, step = 601 (0.152 sec)
INFO:tensorflow:global_step/sec: 666.368
INFO:tensorflow:loss = 0.15406677, step = 701 (0.147 sec

DNNRegression has RMSE of 84.64368559494616
Just using average = 598.6658512720156 has RMSE of 103.34538537526363


In [51]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_dewp', hidden_units=[20,18,13], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3769d797d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_dewp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_dewp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.50938684 0.5685225  0.52078134 0.45745307 0.5399669  0.57444793
 0.5135475  0.4295686  0.49450785 0.5433124  0.46798426 0.41196185
 0.56338316 0.59016114 0.5576928  0.523501   0.5205563  0.5279618
 0.5717111  0.51154953 0.5779994  0.4795633  0.5155359  0.54478365
 0.5579539  0.4238928  0.53108174 0.45774823 0.5030342  0.49067026
 0.5140694  0.46040016 0.54991347 0.46243793 0.5543969  0.5496438
 0.5213857  0.52973515 0.4575017  0.43058544 0.54442626 0.53164417
 0.42636138 0.50159365 0.49674374 0.50797707 0.51341397 0.5596636
 0.5031479  0.5341795  0.4909659  0.4913934  0.50866395 0.5129605
 0.57824856 0.54449826 0.47830206 0.5375739  0.53882295 0.50271994
 0.41915804 0.48136622 0.5187419  0.43378216 0.5232864  0.52376515
 0.48984224 0.49395853 0.4776786  0.5253554  0.43987757 0.4111541
 0.50176436 0.5331188  0.5518082  0.50495774 0.55524355 0.5566426
 0.50909644 0.5162104  0.56643206 0.44025284 0.48248154 0.4977978
 0.55446845 0.48001963 0.5105465  0.45642215 0.43339306 0.41568977
 0

##Sea Level Pressure(slp)


In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/slp_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [53]:
df_slp_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] != 2012]
df_slp_dnn = df_slp_dnn.loc[df_slp_dnn["year"] < 2020]
cols = df_slp_dnn['NUM_COLLISIONS']
df_slp_dnn = df_slp_dnn.drop(columns=['NUM_COLLISIONS'])
df_slp_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_slp_dnn[:6])
df_slp_dnn.describe()

    year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28  1016.1    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17  1014.8    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25  1021.4    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29   999.4    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20  1015.5    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13  1020.7    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,slp,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,...,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000
mean,2016.000391,15.719765,1016.777221,0.082192,0.084932,0.084540,0.077104,0.084932,0.084932,0.082192,...,0.082192,0.084932,0.082192,0.143249,0.142857,0.142857,0.142857,0.142857,0.142466,599.147162
std,2.000294,8.796698,7.628429,0.274710,0.278834,0.278251,0.266808,0.278834,0.278834,0.274710,...,0.274710,0.278834,0.274710,0.350395,0.349996,0.349996,0.349996,0.349996,0.349596,100.268048
min,2013.000000,1.000000,989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,1012.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,1016.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,1021.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,1044.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [54]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_slp_dnn.iloc[np.random.permutation(len(df_slp_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da     slp  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
228   2015  25  1000.4    0    0    0    0    1    0    0  ...    0    0    0   
225   2019   7  1024.8    0    0    0    0    1    0    0  ...    0    0    0   
812   2019  16  1009.9    0    0    0    0    0    0    0  ...    0    0    0   
2514  2013  25  1012.3    0    0    0    0    0    0    0  ...    0    0    0   
3100  2015  18  1034.9    0    0    0    0    0    0    0  ...    0    1    0   
187   2014  23  1014.4    0    0    0    0    1    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
228     0    0    1    0    0    0    0  
225     0    0    0    1    0    0    0  
812     0    0    0    0    0    0    0  
2514    1    0    0    0    0    1    0  
3100    0    0    0    0    0    1    0  
187     0    0    0    0    0    0    1  

[6 rows x 21 columns]


In [55]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

228     406
225     584
812     550
2514    539
3100    628
187     612
Name: NUM_COLLISIONS, dtype: int64


In [56]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [57]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_slp', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3769de7c50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_slp/model.ckpt.
INFO:tensorflow:loss = 519866.2, step = 1
INFO:tensorflow:global_step/sec: 375.058
INFO:tensorflow:loss = 0.007341341, step = 101 (0.275 sec)
INFO:tensorflow:global_step/sec: 477.047
INFO:tensorflow:loss = 0.007361635, step = 201 (0.209 sec)
INFO:tensorflow:global_step/sec: 386.452
INFO:tensorflow:loss = 0.0076099923, step = 301 (0.258 sec)
INFO:tensorflow:global_step/sec: 405.061
INFO:tensorflow:loss = 0.008149187, step = 401 (0.243 sec)
INFO:tensorflow:global_step/sec: 460.494
INFO:tensorflow:loss = 0.007698088, step = 501 (0.221 sec)
INFO:tensorflow:global_step/sec: 389.37
INFO:tensorflow:loss = 0.0068629105, step = 601 (0.257 sec)
INFO:tensorflow:global_step/sec: 388.192
INFO:tensorflow:loss = 0.0063186768, step = 701 (0.254 s

DNNRegression has RMSE of 98.13725359927167
Just using average = 598.7358121330724 has RMSE of 97.87847068956921


In [58]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_slp', hidden_units=[19,15,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d57aa90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_slp', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_slp/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.5103302  0.5103613  0.5070674  0.5101613  0.5117183  0.5106867
 0.51136506 0.50941634 0.5122532  0.5127063  0.5136491  0.5104544
 0.5118376  0.50883657 0.51362693 0.5069814  0.5093562  0.51150364
 0.50702965 0.51470375 0.5106147  0.5130661  0.51313823 0.5104656
 0.5071237  0.5116045  0.5127797  0.510921   0.5125872  0.50882584
 0.51363844 0.50987655 0.51251173 0.5134938  0.5119828  0.50744396
 0.51180637 0.5075727  0.509041   0.5104948  0.51262087 0.5057461
 0.5077617  0.507396   0.5068128  0.5083421  0.5093314  0.509261
 0.50928867 0.5083774  0.51099455 0.5086956  0.50989336 0.51573765
 0.5075681  0.50841147 0.5132842  0.5070294  0.508884   0.5094659
 0.51422936 0.51165926 0.5093714  0.5123597  0.50597346 0.5075458
 0.5059107  0.5125922  0.51007754 0.5091987  0.51165634 0.51374996
 0.5067896  0.5112192  0.51398325 0.50664747 0.50909543 0.5069687
 0.5133351  0.5100954  0.5138924  0.51175046 0.5086712  0.5089871
 0.5086598  0.510961   0.5077289  0.50772566 0.5069042  0.50754243
 0.51

##Gust

In [59]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/gust_clean_dnn.csv', index_col=0, )
print(df[:6])

    year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
3   2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0  20.0   
11  2020  15             508  43.9  38.3  1019.4    8.2   5.4   14.0  15.0   
12  2021   1             257  39.6  29.3  1029.3   10.0   7.6   14.0  20.0   
14  2022  25             235  41.6  31.8  1013.2   10.0   9.6   15.0  19.0   
18  2021   3             186  41.1  32.3  1018.0   10.0  10.3   19.0  27.0   
19  2020   2             413  39.6  28.9  1011.8   10.0  13.0   19.0  26.0   

    ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
3   ...    0    0    0    0    0    0    0    1    0    0  
11  ...    0    0    0    0    0    0    0    0    1    0  
12  ...    0    0    0    0    0    0    0    1    0    0  
14  ...    0    0    0    0    1    0    0    0    0    0  
18  ...    0    0    0    0    0    1    0    0    0    0  
19  ...    0    0    0    0    0    0    0    0    0    1  

[6 rows x 38 columns]


In [60]:
df_gust_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','mxpsd','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] != 2012]
df_gust_dnn = df_gust_dnn.loc[df_gust_dnn["year"] < 2020]
cols = df_gust_dnn['NUM_COLLISIONS']
df_gust_dnn = df_gust_dnn.drop(columns=['NUM_COLLISIONS'])
df_gust_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_gust_dnn[:6])
df_gust_dnn.describe()

    year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
74  2016  17  18.1    0    0    0    0    1    0    0  ...    0    0    0   
76  2014   9  20.0    0    0    0    0    1    0    0  ...    0    0    0   
79  2019  19  21.0    0    0    0    0    1    0    0  ...    0    0    0   
80  2015  11  17.1    0    0    0    0    1    0    0  ...    0    0    0   
83  2015  29  20.0    0    0    0    0    1    0    0  ...    0    0    0   
85  2019  13  15.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
74    0    1    0    0    0    0             451  
76    0    0    0    0    0    1             561  
79    0    0    0    0    0    0             479  
80    0    1    0    0    0    0             341  
83    0    0    0    0    0    1             519  
85    0    1    0    0    0    0             374  

[6 rows x 22 columns]


,year,da,gust,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1629.00000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,...,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000,1629.000000
mean,2015.91283,15.702885,27.511602,0.095764,0.042357,0.104359,0.095150,0.108656,0.046041,0.061387,...,0.096378,0.087784,0.071209,0.139963,0.141191,0.139963,0.151627,0.138122,0.145488,596.513198
std,2.01341,8.667634,7.366770,0.294358,0.201465,0.305819,0.293513,0.311302,0.209637,0.240113,...,0.295200,0.283067,0.257253,0.347055,0.348325,0.347055,0.358769,0.345133,0.352700,104.479660
min,2013.00000,1.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.00000,8.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,526.000000
50%,2016.00000,16.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,597.000000
75%,2018.00000,23.000000,31.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,663.000000
max,2019.00000,31.000000,71.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [61]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_gust_dnn.iloc[np.random.permutation(len(df_gust_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  gust  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
281   2018   6  39.0    0    0    0    0    1    0    0  ...    0    0    0   
977   2018  23  15.9    1    0    0    0    0    0    0  ...    0    0    0   
3204  2013  15  20.0    0    0    0    0    0    0    0  ...    0    1    0   
3013  2014  24  29.9    0    0    0    0    0    0    0  ...    0    0    1   
2433  2018  31  24.1    0    1    0    0    0    0    0  ...    0    0    0   
3601  2019  20  28.9    0    0    1    0    0    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
281     0    0    0    0    0    0    0  
977     0    0    0    1    0    0    0  
3204    0    0    0    0    1    0    0  
3013    0    0    0    0    1    0    0  
2433    0    0    0    0    1    0    0  
3601    0    0    0    0    1    0    0  

[6 rows x 21 columns]


In [62]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

281     599
977     675
3204    617
3013    669
2433    623
3601    698
Name: NUM_COLLISIONS, dtype: int64


In [63]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [64]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_gust', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376dac9a90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_gust/model.ckpt.
INFO:tensorflow:loss = 18673.336, step = 1
INFO:tensorflow:global_step/sec: 484.729
INFO:tensorflow:loss = 0.70193785, step = 101 (0.212 sec)
INFO:tensorflow:global_step/sec: 624.397
INFO:tensorflow:loss = 0.8945514, step = 201 (0.159 sec)
INFO:tensorflow:global_step/sec: 655.731
INFO:tensorflow:loss = 1.0413133, step = 301 (0.151 sec)
INFO:tensorflow:global_step/sec: 694.419
INFO:tensorflow:loss = 0.9185983, step = 401 (0.144 sec)
INFO:tensorflow:global_step/sec: 605.568
INFO:tensorflow:loss = 0.6781819, step = 501 (0.168 sec)
INFO:tensorflow:global_step/sec: 716.529
INFO:tensorflow:loss = 0.47958076, step = 601 (0.139 sec)
INFO:tensorflow:global_step/sec: 711.243
INFO:tensorflow:loss = 0.49747148, step = 701 (0.139 sec)
INFO:te

DNNRegression has RMSE of 783.970272843907
Just using average = 595.926323867997 has RMSE of 101.48823712997127


In [65]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d471990>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


326
326


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[1.194391  1.1049479 1.1870203 1.1668943 1.040269  1.2286973 1.1888226
 1.1962259 1.1737874 1.2359042 1.1580473 1.1932143 1.2301476 1.2346545
 1.1978226 1.2110507 1.1685503 1.1629512 1.1595644 1.2015777 1.2566245
 1.1113663 1.2151308 1.1686465 1.1777934 1.1814578 1.2395948 1.2205758
 1.2178271 1.248207  1.255397  1.2170538 1.0982308 1.2237643 1.1792283
 1.2652383 1.2003063 1.2181973 1.2123485 1.2580627 1.1765153 1.1615616
 1.1081288 1.1996698 1.2026939 1.1819993 1.0811682 1.1169717 1.2250378
 1.1825143 1.1953098 1.235367  1.2359048 1.0429051 1.2581121 1.2040194
 1.185708  1.1822493 1.2109714 1.2125906 1.2258148 1.162156  1.1766837
 1.1772774 1.2070302 1.2180911 1.1651337 1.0667249 1.2588843 1.0631528
 1.2660097 1.1971905 1.2171893 1.1015611 1.2190645 1.1461064 1.2118015
 1.1074315 1.2208043 1.1856952 1.2239091 1.2390212 1.1499317 1.2235936
 1.231025  1.1850772 1.1791211 1.1788703 1.2213271 1.2305825 1.2382582
 1.1406907 1.1323256 1.182517  1.0485505 1.2256471 1.2203379 1.0457914
 1.218

##Maximum Sustained Wind Speed (mxpsd)

In [66]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/mxpsd_clean_dnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [67]:
df_mxpsd_dnn = df.drop(columns=['temp', 'prcp', 'dewp','visib','max','min','sndp','wdsp','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] != 2012]
df_mxpsd_dnn = df_mxpsd_dnn.loc[df_mxpsd_dnn["year"] < 2020]
cols = df_mxpsd_dnn['NUM_COLLISIONS']
df_mxpsd_dnn = df_mxpsd_dnn.drop(columns=['NUM_COLLISIONS'])
df_mxpsd_dnn.insert(loc=21, column='NUM_COLLISIONS', value=cols)
print(df_mxpsd_dnn[:6])
df_mxpsd_dnn.describe()

    year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Nov  Oct  Sep  \
49  2016  28    8.9    0    0    0    0    1    0    0  ...    0    0    0   
51  2014  17    8.9    0    0    0    0    1    0    0  ...    0    0    0   
54  2016  25    8.9    0    0    0    0    1    0    0  ...    0    0    0   
55  2016  29    9.9    0    0    0    0    1    0    0  ...    0    0    0   
58  2017  20    9.9    0    0    0    0    1    0    0  ...    0    0    0   
59  2013  13    9.9    0    0    0    0    1    0    0  ...    0    0    0   

    Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    1             681  
51    0    0    0    1    0    0             589  
54    0    0    1    0    0    0             658  
55    0    0    0    1    0    0             645  
58    0    0    0    1    0    0             605  
59    0    1    0    0    0    0             373  

[6 rows x 22 columns]


,year,da,mxpsd,Apr,Aug,Dec,Feb,Jan,Jul,Jun,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,...,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000,2553.000000
mean,2016.001567,15.737172,17.240110,0.082256,0.084998,0.084998,0.077164,0.084998,0.084998,0.082256,...,0.081864,0.084998,0.081473,0.143361,0.142969,0.142969,0.142969,0.142577,0.142186,599.033686
std,2.000587,8.797367,5.858333,0.274808,0.278933,0.278933,0.266904,0.278933,0.278933,0.274808,...,0.274212,0.278933,0.273613,0.350509,0.350110,0.350110,0.350110,0.349710,0.349309,100.284761
min,2013.000000,1.000000,5.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.000000,8.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.000000,16.000000,15.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.000000,23.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.000000,31.000000,49.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [68]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_mxpsd_dnn.iloc[np.random.permutation(len(df_mxpsd_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  mxpsd  Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  May  Nov  Oct  \
1687  2019  10   14.0    0    0    0    0    0    0    1  ...    0    0    0   
1463  2019  29   20.0    0    0    0    0    0    0    0  ...    1    0    0   
2638  2014  24   15.0    0    0    0    0    0    0    0  ...    0    0    0   
2641  2018  18   15.9    0    0    0    0    0    0    0  ...    0    0    0   
1093  2017  21   19.0    1    0    0    0    0    0    0  ...    0    0    0   
238   2015   4   22.9    0    0    0    0    1    0    0  ...    0    0    0   

      Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1687    0    0    0    1    0    0    0  
1463    0    0    0    0    0    1    0  
2638    1    0    0    0    0    1    0  
2641    1    1    0    0    0    0    0  
1093    0    0    0    0    1    0    0  
238     0    0    1    0    0    0    0  

[6 rows x 21 columns]


In [69]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1687    613
1463    601
2638    566
2641    740
1093    690
238     381
Name: NUM_COLLISIONS, dtype: int64


In [70]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)

21


In [71]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_mxpsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_mxpsd', hidden_units=[19,15,11], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f376d97df10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_mxpsd', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_mxpsd/model.ckpt.
INFO:tensorflow:loss = 75043.26, step = 1
INFO:tensorflow:global_step/sec: 519.179
INFO:tensorflow:loss = 1.2256441, step = 101 (0.198 sec)
INFO:tensorflow:global_step/sec: 659.017
INFO:tensorflow:loss = 0.0972218, step = 201 (0.151 sec)
INFO:tensorflow:global_step/sec: 693.309
INFO:tensorflow:loss = 0.10458371, step = 301 (0.146 sec)
INFO:tensorflow:global_step/sec: 649.582
INFO:tensorflow:loss = 0.08815303, step = 401 (0.152 sec)
INFO:tensorflow:global_step/sec: 736.557
INFO:tensorflow:loss = 0.08122876, step = 501 (0.137 sec)
INFO:tensorflow:global_step/sec: 725.304
INFO:tensorflow:loss = 0.0872712, step = 601 (0.139 sec)
INFO:tensorflow:global_step/sec: 697.915
INFO:tensorflow:loss = 0.09661062, step = 701 (0.142 sec)
INFO:t

DNNRegression has RMSE of 325.02715731466327
Just using average = 599.409402546523 has RMSE of 99.09077607746597


In [72]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_gust', hidden_units=[23,19,11], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3769d79ad0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_gust', '_session_creation_timeout_secs': 7200}


511
511


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_gust/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[1.2263927 1.2170818 1.0584722 1.1852422 1.1661063 1.1748366 1.2012285
 1.178473  1.2680091 1.2221575 1.0659655 1.1959519 1.1773702 1.2193909
 1.2336334 1.1494995 1.2073975 1.2269225 1.2524749 1.2465492 1.229411
 1.1820177 1.1855026 1.0962048 1.0685252 1.1862141 1.0814662 1.0950677
 1.1849273 1.1802145 1.2982483 1.1581275 1.2647433 1.2013193 1.2174778
 1.2352034 1.2129431 1.2095284 1.2132425 1.1360071 1.1800944 1.1714175
 1.1665826 1.1960803 1.2428361 1.187238  1.1788391 1.2150774 1.1497185
 1.1874614 1.1245847 1.2578483 1.1063821 1.2624607 1.222285  1.2148796
 1.1466534 1.2918048 1.2542887 1.1718099 1.1917621 1.2107341 1.2685
 1.2201335 1.167687  1.2410557 1.1757586 1.1785907 1.206907  1.2020473
 1.1034303 1.1578122 1.1723841 1.2535187 1.261845  1.2066633 1.2123258
 1.159027  1.251253  1.2218122 1.2133411 1.2161541 1.2226516 1.1764673
 1.184691  1.2148336 1.245418  1.0459018 1.2188526 1.2181077 1.0419735
 1.0776452 1.1763196 1.1878501 1.183058  1.2241572 1.2212137 1.0787548
 1.2965889

##Others

In [73]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/datadnn.csv', index_col=0, )
print(df[:6])

   year  da  NUM_COLLISIONS  temp  dewp     slp  visib  wdsp  mxpsd   gust  \
1  2020  24             524  37.3  33.7  1028.5    6.5   3.3    8.0  999.9   
2  2021  12             278  37.0  29.1  1019.0   10.0   6.5   12.0  999.9   
3  2021  22             254  36.5  28.4  1003.1   10.0   7.8   12.0   20.0   
4  2021  27             262  34.6  33.8  1012.8    8.0   7.8   12.0  999.9   
5  2021  26             263  31.9  23.4  1016.9    9.0   7.4   12.0  999.9   
6  2022  24             237  34.5  23.8  1010.6    9.7   7.4   12.0  999.9   

   ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  ...    0    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    1    0    0    0    0    0  
3  ...    0    0    0    0    0    0    0    1    0    0  
4  ...    0    0    0    0    0    0    0    0    1    0  
5  ...    0    0    0    0    1    0    0    0    0    0  
6  ...    0    0    0    0    0    0    1    0    0    0  

[6 rows x 38 columns]


In [74]:
dnn = df.drop(columns=[ 'prcp', 'dewp','mxpsd','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
dnn = dnn.loc[dnn["year"] != 2012]
dnn = dnn.loc[dnn["year"] < 2020]
cols = dnn['NUM_COLLISIONS']
dnn = dnn.drop(columns=['NUM_COLLISIONS'])
dnn.insert(loc=26, column='NUM_COLLISIONS', value=cols)
print(dnn[:6])
dnn.describe()

    year  da  temp  visib  wdsp   max   min   sndp  Apr  Aug  ...  Nov  Oct  \
49  2016  28  35.0   10.0   4.3  46.0  23.0  999.9    0    0  ...    0    0   
51  2014  17  38.6    6.7   3.7  44.1  32.0  999.9    0    0  ...    0    0   
54  2016  25  33.5   10.0   6.5  37.9  30.0  999.9    0    0  ...    0    0   
55  2016  29  41.3   10.0   5.9  45.0  23.0  999.9    0    0  ...    0    0   
58  2017  20  39.9   10.0   4.3  45.0  37.0  999.9    0    0  ...    0    0   
59  2013  13  45.4    4.3   5.8  46.9  44.1  999.9    0    0  ...    0    0   

    Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
49    0    0    0    0    0    0    1             681  
51    0    0    0    0    1    0    0             589  
54    0    0    0    1    0    0    0             658  
55    0    0    0    0    1    0    0             645  
58    0    0    0    0    1    0    0             605  
59    0    0    1    0    0    0    0             373  

[6 rows x 27 columns]


,year,da,temp,visib,wdsp,max,min,sndp,Apr,Aug,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,2556.0,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2.556000e+03,2556.000000,2556.000000,...,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000,2556.000000
mean,2016.0,15.725743,51.487715,8.295618,10.682864,59.564280,43.869757,9.999000e+02,0.082160,0.084898,...,0.082160,0.084898,0.082160,0.143192,0.142801,0.142801,0.142801,0.142801,0.142801,599.118936
std,2.0,8.800168,14.162738,2.207870,4.242687,14.279867,14.722751,2.274182e-13,0.274661,0.278785,...,0.274661,0.278785,0.274661,0.350338,0.349939,0.349939,0.349939,0.349939,0.349939,100.258581
min,2013.0,1.000000,5.800000,0.200000,2.700000,18.000000,-2.000000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2014.0,8.000000,40.400000,7.100000,7.700000,48.000000,33.100000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,2016.0,16.000000,52.000000,9.400000,10.100000,60.100000,44.100000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,2018.0,23.000000,63.900000,10.000000,13.000000,72.000000,55.900000,9.999000e+02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,665.000000
max,2019.0,31.000000,77.500000,10.000000,39.300000,90.000000,71.600000,9.999000e+02,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [75]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = dnn.iloc[np.random.permutation(len(dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      year  da  temp  visib  wdsp   max   min   sndp  Apr  Aug  ...  May  Nov  \
1702  2019  30  69.3    8.3   9.4  77.0  61.0  999.9    0    0  ...    0    0   
2054  2013  27  69.1    9.6   8.8  73.9  64.9  999.9    0    0  ...    0    0   
730   2016   7  36.8    9.9   9.0  46.0  23.0  999.9    0    0  ...    0    0   
2625  2014   9  64.6   10.0  10.9  73.0  55.9  999.9    0    0  ...    0    0   
1340  2014  18  56.5    9.8   8.5  63.0  50.0  999.9    0    0  ...    1    0   
1961  2015   2  68.7    7.1   7.0  77.0  61.0  999.9    0    0  ...    0    0   

      Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1702    0    0    0    1    0    0    0    0  
2054    0    0    0    0    0    0    0    0  
730     0    0    0    0    1    0    0    0  
2625    0    1    1    0    0    0    0    0  
1340    0    0    0    1    0    0    0    0  
1961    0    0    0    0    0    0    0    1  

[6 rows x 26 columns]


In [76]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1702    569
2054    516
730     571
2625    592
1340    507
1961    603
Name: NUM_COLLISIONS, dtype: int64


In [77]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


26


In [78]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,13,9,7], optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3769e0d210>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 30449.512, step = 1
INFO:tensorflow:global_step/sec: 495.037
INFO:tensorflow:loss = 0.25086278, step = 101 (0.207 sec)
INFO:tensorflow:global_step/sec: 647.691
INFO:tensorflow:loss = 0.23757836, step = 201 (0.155 sec)
INFO:tensorflow:global_step/sec: 576.943
INFO:tensorflow:loss = 0.25054714, step = 301 (0.172 sec)
INFO:tensorflow:global_step/sec: 667.635
INFO:tensorflow:loss = 0.25477073, step = 401 (0.147 sec)
INFO:tensorflow:global_step/sec: 648.086
INFO:tensorflow:loss = 0.23914641, step = 501 (0.158 sec)
INFO:tensorflow:global_step/sec: 678.407
INFO:tensorflow:loss = 0.24517313, step = 601 (0.148 sec)
INFO:tensorflow:global_step/sec: 650.553
INFO:tensorflow:loss = 0.23849544, step = 701 (0.150 sec)
INFO:ten

DNNRegression has RMSE of 98.91113368524593
Just using average = 598.5019569471624 has RMSE of 98.91593876158191


In [79]:
print(predictors[trainsize:].values)
#print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model', hidden_units=[17,13,9,7], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f37698d21d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model', '_session_creation_timeout_secs': 7200}


[[2.014e+03 2.100e+01 6.610e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.014e+03 1.700e+01 6.830e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.018e+03 2.400e+01 6.710e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 ...
 [2.018e+03 5.000e+00 6.220e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 [2.016e+03 2.000e+00 6.130e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.015e+03 1.900e+01 7.140e+01 ... 0.000e+00 0.000e+00 0.000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642 0.515642
 0.515642 0.515642 0.515642 0.515642 0.515642 0.515

## Location


REF https://www.geeksforgeeks.org/read-a-zipped-file-as-a-pandas-dataframe/ 12/11

In [80]:
df_loc = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/locdnn.zip', index_col=0,compression='zip' )
print(df_loc[:6])

   year  da  NUM_COLLISIONS   latitude  longitude  temp  dewp     slp  visib  \
1  2018   2               1  40.681750 -73.967480  14.7   2.0  1024.9   10.0   
2  2018   2               1  40.645370 -73.945110  14.7   2.0  1024.9   10.0   
3  2018   2               1  40.614830 -73.998380  14.7   2.0  1024.9   10.0   
4  2018   2               1  40.592190 -74.087395  14.7   2.0  1024.9   10.0   
5  2018   2               1  40.769817 -73.782370  14.7   2.0  1024.9   10.0   
6  2018   2               1  40.660175 -73.928200  14.7   2.0  1024.9   10.0   

   wdsp  ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
1  12.9  ...    0    0    0    0    1    0    0    0    0    0  
2  12.9  ...    0    0    0    0    1    0    0    0    0    0  
3  12.9  ...    0    0    0    0    1    0    0    0    0    0  
4  12.9  ...    0    0    0    0    1    0    0    0    0    0  
5  12.9  ...    0    0    0    0    1    0    0    0    0    0  
6  12.9  ...    0    0    0    0    1    0    0  

In [81]:
df_loc_dnn = df_loc.drop(columns=[ 'prcp', 'dewp','mxpsd','gust','slp','thunder','tornado_funnel_cloud','fog','rain_drizzle','snow_ice_pellets','hail'])
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] != 2012]
df_loc_dnn = df_loc_dnn.loc[df_loc_dnn["year"] < 2020]
cols = df_loc_dnn['NUM_COLLISIONS']
df_loc_dnn = df_loc_dnn.drop(columns=['NUM_COLLISIONS'])
df_loc_dnn.insert(loc=28, column='NUM_COLLISIONS', value=cols)
print(df_loc_dnn[:6])
df_loc_dnn.describe()

   year  da   latitude  longitude  temp  visib  wdsp   max  min   sndp  ...  \
1  2018   2  40.681750 -73.967480  14.7   10.0  12.9  18.0  5.0  999.9  ...   
2  2018   2  40.645370 -73.945110  14.7   10.0  12.9  18.0  5.0  999.9  ...   
3  2018   2  40.614830 -73.998380  14.7   10.0  12.9  18.0  5.0  999.9  ...   
4  2018   2  40.592190 -74.087395  14.7   10.0  12.9  18.0  5.0  999.9  ...   
5  2018   2  40.769817 -73.782370  14.7   10.0  12.9  18.0  5.0  999.9  ...   
6  2018   2  40.660175 -73.928200  14.7   10.0  12.9  18.0  5.0  999.9  ...   

   Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  NUM_COLLISIONS  
1    0    0    0    1    0    0    0    0    0               1  
2    0    0    0    1    0    0    0    0    0               1  
3    0    0    0    1    0    0    0    0    0               1  
4    0    0    0    1    0    0    0    0    0               1  
5    0    0    0    1    0    0    0    0    0               1  
6    0    0    0    1    0    0    0    0    0          

,year,da,latitude,longitude,temp,visib,wdsp,max,min,sndp,...,Nov,Oct,Sep,Mon,Sat,Sun,Thu,Tue,Wed,NUM_COLLISIONS
count,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,...,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06,1.311775e+06
mean,2.016149e+03,1.566747e+01,4.072402e+01,-7.392063e+01,5.203610e+01,8.262622e+00,1.066076e+01,6.012695e+01,4.442143e+01,9.999000e+02,...,8.510758e-02,8.877971e-02,8.587448e-02,1.487626e-01,1.173738e-01,1.438814e-01,1.597195e-01,1.467698e-01,1.505395e-01,1.026582e+00
std,1.970562e+00,8.752606e+00,7.845179e-02,8.651997e-02,1.410404e+01,2.220453e+00,4.197681e+00,1.424273e+01,1.468128e+01,2.842172e-12,...,2.790418e-01,2.844256e-01,2.801787e-01,3.558544e-01,3.218653e-01,3.509695e-01,3.663458e-01,3.538765e-01,3.575996e-01,1.784039e-01
min,2.013000e+03,1.000000e+00,4.049895e+01,-7.425453e+01,5.800000e+00,2.000000e-01,2.700000e+00,1.800000e+01,-2.000000e+00,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,2.014000e+03,8.000000e+00,4.066895e+01,-7.397644e+01,4.100000e+01,7.000000e+00,7.700000e+00,4.890000e+01,3.310000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,2.016000e+03,1.600000e+01,4.072234e+01,-7.392891e+01,5.300000e+01,9.300000e+00,1.010000e+01,6.100000e+01,4.500000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,2.018000e+03,2.300000e+01,4.076838e+01,-7.386641e+01,6.430000e+01,1.000000e+01,1.290000e+01,7.300000e+01,5.700000e+01,9.999000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,2.019000e+03,3.100000e+01,4.091288e+01,-7.366301e+01,7.750000e+01,1.000000e+01,3.930000e+01,9.000000e+01,7.160000e+01,9.999000e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01


In [82]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_loc_dnn.iloc[np.random.permutation(len(df_loc_dnn))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

         year  da   latitude  longitude  temp  visib  wdsp   max   min   sndp  \
267720   2014   2  40.543154 -74.173772  39.3    5.5   8.7  45.0  30.9  999.9   
1016167  2019   9  40.801125 -73.929880  73.7    8.9  10.4  78.1  69.1  999.9   
856337   2017   2  40.756490 -73.997765  58.0   10.0   9.3  66.0  48.0  999.9   
1075328  2015  23  40.709821 -73.940196  71.9   10.0   8.3  80.1  63.0  999.9   
267094   2018  26  40.704200 -73.964290  36.4   10.0  19.4  45.0  32.0  999.9   
185493   2014  28  40.719607 -73.730291  41.2    8.9  13.7  48.0  34.0  999.9   

         ...  May  Nov  Oct  Sep  Mon  Sat  Sun  Thu  Tue  Wed  
267720   ...    0    0    0    0    0    1    0    0    0    0  
1016167  ...    0    0    0    0    0    0    0    1    0    0  
856337   ...    0    0    1    0    0    0    1    0    0    0  
1075328  ...    0    0    0    0    0    0    0    0    0    1  
267094   ...    0    0    0    0    0    0    1    0    0    0  
185493   ...    0    0    0    0    0    0

In [83]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

267720     1
1016167    2
856337     1
1075328    1
267094     1
185493     1
Name: NUM_COLLISIONS, dtype: int64


In [84]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

# Define the number of output values (targets)
noutputs = 1
# Define the number of input values (predictors)
nppredictors = len(shuffle.columns) - noutputs
print(nppredictors)


28


In [85]:

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/DNN_regression_trained_model_loc', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[16,7,3], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_COLLISIONS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3769a7ae10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


starting to train


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/DNN_regression_trained_model_loc/model.ckpt.
INFO:tensorflow:loss = 156.72598, step = 1
INFO:tensorflow:global_step/sec: 555.74
INFO:tensorflow:loss = 0.0012783015, step = 101 (0.185 sec)
INFO:tensorflow:global_step/sec: 716.757
INFO:tensorflow:loss = 0.000111670524, step = 201 (0.139 sec)
INFO:tensorflow:global_step/sec: 683.106
INFO:tensorflow:loss = 4.7639082e-06, step = 301 (0.147 sec)
INFO:tensorflow:global_step/sec: 685.023
INFO:tensorflow:loss = 9.176864e-08, step = 401 (0.144 sec)
INFO:tensorflow:global_step/sec: 692.376
INFO:tensorflow:loss = 2.3307496e-08, step = 501 (0.144 sec)
INFO:tensorflow:global_step/sec: 692.66
INFO:tensorflow:loss = 1.1534198e-08, step = 601 (0.146 sec)
INFO:tensorflow:global_step/sec: 679.172
INFO:tensorflow:loss = 1.1537262e-08, step = 70

DNNRegression has RMSE of 0.18042282495495965
Just using average = 1.026596596215052 has RMSE of 0.17866213280656273


In [86]:
print(predictors[trainsize:].values)
#print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_loc', hidden_units=[16,7,3], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f37711e7a90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_loc', '_session_creation_timeout_secs': 7200}


[[2.01300000e+03 2.30000000e+01 4.07660506e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01800000e+03 1.30000000e+01 4.07310680e+01 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.01700000e+03 1.00000000e+00 4.05991440e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.01800000e+03 4.00000000e+00 4.07605360e+01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.01800000e+03 1.40000000e+01 4.07658650e+01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.01700000e+03 1.60000000e+01 4.07441000e+01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]]


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_loc/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[0.00086252 0.00086252 0.00086252 ... 0.00086252 0.00086252 0.00086252]
[0.00086133 0.00086133 0.00086133 ... 0.00086133 0.00086133 0.00086133]
The trained model has an aproximate error rate of 0.025144460959203294 which equates to 2%
